## Deliverable 2. Create a Customer Travel Destinations Map.

In [38]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [39]:
! pwd && ls -alsF

/Users/bradroberts/Desktop/Module_6/Vacation_Search
total 1776
   0 drwxr-xr-x@ 10 bradroberts  staff     320 Feb  6 20:37 ./
   0 drwxr-xr-x@ 15 bradroberts  staff     480 Feb  6 20:37 ../
  24 -rw-r--r--@  1 bradroberts  staff   10244 Feb  6 18:28 .DS_Store
   0 drwxr-xr-x@  3 bradroberts  staff      96 Feb  6 20:02 .ipynb_checkpoints/
 192 -rw-r--r--@  1 bradroberts  staff   97889 Feb  6 20:02 Vacation_Search.ipynb
   8 -rw-r--r--   1 bradroberts  staff    2100 Feb  6 18:18 WeatherPy_vacation.csv
1544 -rw-r--r--@  1 bradroberts  staff  788659 Feb  6 15:15 WeatherPy_vacation_map.png
   0 drwxr-xr-x@  8 bradroberts  staff     256 Feb  6 20:37 Weather_Database/
   0 drwxr-xr-x@  4 bradroberts  staff     128 Feb  6 16:08 __pycache__/
   8 -rw-r--r--   1 bradroberts  staff     175 Feb  6 13:26 config.py


In [41]:
# 1. Import the WeatherPy_Database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head(10)



,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Tangwani,PK,28.2787,68.9972,53.53,38,79,11.99,broken clouds
1,1,Hermanus,ZA,-34.4187,19.2345,63.90,93,91,3.22,overcast clouds
2,2,Tuatapere,NZ,-46.1333,167.6833,59.83,45,0,6.82,clear sky
3,3,Mataura,NZ,-46.1927,168.8643,59.49,36,21,1.88,few clouds
4,4,Bethel,US,41.3712,-73.4140,27.64,75,0,3.44,clear sky
5,5,Yellowknife,CA,62.4560,-114.3525,-1.68,84,75,4.61,light snow
6,6,Lasa,IT,46.6166,10.7002,44.06,86,100,5.79,light snow
7,7,Punta Arenas,CL,-53.1500,-70.9167,51.91,87,75,6.91,broken clouds
8,8,Rikitea,PF,-23.1203,-134.9692,79.00,71,45,19.26,scattered clouds
9,9,Puerto Ayora,EC,-0.7393,-90.3518,73.54,92,15,3.00,few clouds


In [42]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [44]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
8,8,Rikitea,PF,-23.1203,-134.9692,79.00,71,45,19.26,scattered clouds
9,9,Puerto Ayora,EC,-0.7393,-90.3518,73.54,92,15,3.00,few clouds
10,10,Atuona,PF,-9.8000,-139.0333,78.91,74,12,18.68,few clouds
11,11,Mbandaka,CD,0.0487,18.2603,73.38,40,71,2.28,broken clouds
14,14,Puerto Penasco,MX,31.3167,-113.5333,70.92,12,0,5.99,clear sky
17,17,Vaini,TO,-21.2000,-175.2000,78.96,94,100,12.66,light rain
19,19,Sibu,MY,2.3000,111.8167,77.31,88,40,5.75,scattered clouds
21,21,Dubbo,AU,-32.2500,148.6167,73.53,49,100,17.27,overcast clouds
22,22,Kavieng,PG,-2.5744,150.7967,83.79,71,89,2.24,overcast clouds
24,24,Hilo,US,19.7297,-155.0900,75.25,89,80,1.99,broken clouds


In [45]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.notnull().sum()

City_ID                241
City                   241
Country                240
Lat                    241
Lng                    241
Max Temp               241
Humidity               241
Cloudiness             241
Wind Speed             241
Current Description    241
dtype: int64

In [46]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna(axis=1, how='all')


In [47]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(20)



,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Rikitea,PF,79.00,scattered clouds,-23.1203,-134.9692,
9,Puerto Ayora,EC,73.54,few clouds,-0.7393,-90.3518,
10,Atuona,PF,78.91,few clouds,-9.8000,-139.0333,
11,Mbandaka,CD,73.38,broken clouds,0.0487,18.2603,
14,Puerto Penasco,MX,70.92,clear sky,31.3167,-113.5333,
17,Vaini,TO,78.96,light rain,-21.2000,-175.2000,
19,Sibu,MY,77.31,scattered clouds,2.3000,111.8167,
21,Dubbo,AU,73.53,overcast clouds,-32.2500,148.6167,
22,Kavieng,PG,83.79,overcast clouds,-2.5744,150.7967,
24,Hilo,US,75.25,broken clouds,19.7297,-155.0900,


In [48]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
hotel_df.head(20)

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Rikitea,PF,79.00,scattered clouds,-23.1203,-134.9692,People ThankYou
9,Puerto Ayora,EC,73.54,few clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
10,Atuona,PF,78.91,few clouds,-9.8000,-139.0333,Villa Enata
11,Mbandaka,CD,73.38,broken clouds,0.0487,18.2603,Guest House Epervier
14,Puerto Penasco,MX,70.92,clear sky,31.3167,-113.5333,Playa Bonita Resort
17,Vaini,TO,78.96,light rain,-21.2000,-175.2000,Keleti Beach Resort
19,Sibu,MY,77.31,scattered clouds,2.3000,111.8167,RH HOTEL (4 Star)
21,Dubbo,AU,73.53,overcast clouds,-32.2500,148.6167,Quality Inn Dubbo International
22,Kavieng,PG,83.79,overcast clouds,-2.5744,150.7967,Nusa Island Retreat
24,Hilo,US,75.25,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel


In [49]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.dropna(inplace = True)
hotel_df.head(20)

hotel_df = hotel_df[hotel_df['Hotel Name'] != ""]
hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Rikitea,PF,79.00,scattered clouds,-23.1203,-134.9692,People ThankYou
9,Puerto Ayora,EC,73.54,few clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
10,Atuona,PF,78.91,few clouds,-9.8000,-139.0333,Villa Enata
11,Mbandaka,CD,73.38,broken clouds,0.0487,18.2603,Guest House Epervier
14,Puerto Penasco,MX,70.92,clear sky,31.3167,-113.5333,Playa Bonita Resort
17,Vaini,TO,78.96,light rain,-21.2000,-175.2000,Keleti Beach Resort
19,Sibu,MY,77.31,scattered clouds,2.3000,111.8167,RH HOTEL (4 Star)
21,Dubbo,AU,73.53,overcast clouds,-32.2500,148.6167,Quality Inn Dubbo International
22,Kavieng,PG,83.79,overcast clouds,-2.5744,150.7967,Nusa Island Retreat
24,Hilo,US,75.25,broken clouds,19.7297,-155.0900,Hilo Hawaiian Hotel


In [50]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [51]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [52]:
# 11a. Add a marker layer for each city to the map. 
# Add info_box_content=hotel_info to the gmaps.marker_layer() attribute with the locations.

# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))